# Machine Learning Algorithms Comparitive Analysis

### Importing Libraries

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn as sk
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import  accuracy_score, f1_score, precision_score,confusion_matrix, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

### Importing Dataset

In [ ]:
df = pd.read_csv('/Users/hassanshahzad/Desktop/Westminster/Year3/FinalYearProject/Code/Dataset/diabetes_data_upload.csv')
df.head()

In [ ]:
# Checking for null values
df.isna().sum()

In [ ]:
# Checking for data types
df.info()

# Coverting columns to lower case
df.columns = map(str.lower, df.columns)

### Data Visualization

In [ ]:
# Diabetes distribution by age
plt.figure(figsize=(8,6))
sns.histplot(df['age'],bins=30, color='royalblue', kde=True)  
plt.title('Diabetes distribution by age', fontsize=16);

In [ ]:
# Diabetes distribution by gender
plt.title('Diabetes cases by Gender', fontsize=17)
sns.countplot(x='gender', hue='class', data=df, palette=['#ff8080', '#78ebdc'])

In [ ]:
# Bar plots for symptoms and diabetes cases
count = 1
plt.figure(figsize=(15,20))
plt.suptitle('Correlation between Symptoms and Diabetes Cases'+ '\n', fontsize=20)
for i in df.columns:
    if i not in ['class', 'age', 'gender']:
        plt.subplot(5,4,count)
        plt.title(f'{i.title()}', fontweight='bold', fontsize=14)
        count +=1
        plt.tight_layout()
        total_count = df[i].value_counts()
        total_percentage = total_count / total_count.sum() * 100
        total_percentage.plot(kind="bar", color=['royalblue','lightblue'])
        plt.ylabel('Count')
        plt.title(f'{i.title()}',fontweight='bold',fontsize=12)
        for j, v in enumerate(total_percentage):
            plt.text(j, v, f"{v:.1f}%", ha='center', va='bottom', fontweight='bold')
plt.show()

### Data Preprocessing

In [ ]:
# Changing positive to 1 and negative to 0
df['class'] = df['class'].replace({'Positive': 1, 'Negative': 0})

# Seperating target feature from dataset
X = df.drop('class', axis=1)
y = df['class']

# Storing features to list
objectList = X.select_dtypes(include='object').columns.tolist()
print(objectList)

In [ ]:
# Label Encoding for object to numeric transformation
encoder = preprocessing.LabelEncoder()

for col in X.columns:
    if X[col].dtype=='object':
        X[col]=encoder.fit_transform(X[col])
    
# Printing datatypes of features    
X.info()

In [ ]:
# Printing first 5 rows of dataset
X.head()

In [ ]:
# Correlation between features and diabetes
X.corrwith(y).plot(kind='bar', grid=True, figsize=(8, 4), title="Correlation with Diabetes")

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

### Data Normalization

In [ ]:
minmax = MinMaxScaler()
X_train[['age']] = minmax.fit_transform(X_train[['age']])
X_test[['age']] = minmax.transform(X_test[['age']])
X_train.head()

### Model Building

##### Logistic Regression

In [ ]:
lg = LogisticRegression(random_state=0, penalty='l2')
lg.fit(X_train, y_train)

In [ ]:
# Cross Validation
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
score = 'accuracy'

acc_lg = model_selection.cross_val_score(lg, X_train, y_train, cv=kfold, scoring=score)
acc_lg.mean()

In [ ]:
# Evaluation Metrics
y_pred_lg = lg.predict(X_test)

lg_acc = accuracy_score(y_test, y_pred_lg)
lg_roc = roc_auc_score(y_test, y_pred_lg)
lg_prec = precision_score(y_test, y_pred_lg)
lg_rec = recall_score(y_test, y_pred_lg)
lg_f1 = f1_score(y_test, y_pred_lg)

results_lg = pd.DataFrame([['Logistic Regression', lg_acc, acc_lg.mean(), lg_roc, lg_prec, lg_rec, lg_f1]], columns=['Model', 'Accuracy', 'Cross Val Accuracy', 'ROC', 'Precision', 'Recall', 'F1 Score'])

results_lg

##### Decision Trees Classifier

In [275]:
dt = DecisionTreeClassifier(criterion='gini', random_state=0)
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [ ]:
# Cross Validation
acc_dt = model_selection.cross_val_score(dt, X_train, y_train, cv = kfold, scoring = score)
acc_dt.mean()

In [272]:
# Evaluation Metrics
y_pred_dt = dt.predict(X_test)

dt_acc = accuracy_score(y_test, y_pred_dt)
dt_roc = roc_auc_score(y_test, y_pred_dt)
dt_prec = precision_score(y_test, y_pred_dt)
dt_rec = recall_score(y_test, y_pred_dt)
dt_f1 = f1_score(y_test, y_pred_dt)

results_dt = pd.DataFrame([['Decision Trees Classifier', dt_acc, acc_dt.mean(), dt_roc, dt_prec, dt_rec, dt_f1]], columns=['Model', 'Accuracy', 'Cross Val Accuracy', 'ROC', 'Precision', 'Recall', 'F1 Score'])

results_dt

,Model,Accuracy,Cross Val Accuracy,ROC,Precision,Recall,F1 Score
0,Decision Trees Classifier,0.961538,0.963995,0.960606,0.966667,0.966667,0.966667


##### Random Forest Classifier

In [274]:
estimators = 100

for i in range(1, estimators):
    rf = RandomForestClassifier(n_estimators=i, criterion='entropy', random_state=0)
    rf.fit(X_train, y_train)